In [1]:
import sys, os
from models import FCN_model
from preprocessing import ConstantLengthDataGenerator
import numpy as np
import tensorflow.keras as keras
import matplotlib.pyplot as plt
import sklearn
from sklearn.model_selection import train_test_split
import mlflow
mlflow.set_experiment("FCN")
mlflow.tensorflow.autolog()

In [2]:
data_path = "./data"
X, y = np.load(f"{data_path}/X.npy", allow_pickle=True), np.load(f"{data_path}/y.npy")
y.shape, X.shape

((60555, 1), (60555,))

In [3]:
mask = np.char.startswith(y, "GunPoint").reshape(-1)
y = y[mask, :]
X = X[mask]
mlflow.log_param("y.unique", np.unique(y))

array(['GunPointAgeSpan_1', 'GunPointAgeSpan_2',
       'GunPointMaleVersusFemale_1', 'GunPointMaleVersusFemale_2',
       'GunPointOldVersusYoung_1', 'GunPointOldVersusYoung_2',
       'GunPoint_1', 'GunPoint_2'], dtype='<U42')

In [4]:
y_encoder = sklearn.preprocessing.OneHotEncoder(categories="auto")
y = y_encoder.fit_transform(y.reshape(-1, 1)).toarray()
mlflow.log_param("y.shape", y.shape)
y.shape, X.shape

((456, 8), (456,))

In [5]:
number_of_classes = y.shape[1]
initial_learning_rate = 1e-4
output_directory = f"{data_path}/models/fcn/outputs"
batch_size = min(64, X.shape[0])
os.makedirs(output_directory, exist_ok=True)

In [6]:
input_layer = keras.layers.Input(shape=(None, 1))
fcn_model = FCN_model(number_of_classes=number_of_classes)(input_layer)
model = keras.models.Model(inputs=input_layer, outputs=fcn_model)

lr_schedule = keras.optimizers.schedules.ExponentialDecay(
    initial_learning_rate=initial_learning_rate, decay_steps=3, decay_rate=1
)
model.compile(
    loss="categorical_crossentropy",
    optimizer=keras.optimizers.Adam(lr_schedule),
    metrics=["accuracy"],
    #     run_eagerly=True,
)

In [7]:
model.summary()

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, None, 1)]         0         
                                                                 
 conv1d (Conv1D)             (None, None, 128)         1536      
                                                                 
 batch_normalization (BatchN  (None, None, 128)        512       
 ormalization)                                                   
                                                                 
 activation (Activation)     (None, None, 128)         0         
                                                                 
 conv1d_1 (Conv1D)           (None, None, 256)         164096    
                                                                 
 batch_normalization_1 (Batc  (None, None, 256)        1024      
 hNormalization)                                             

In [8]:
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.25)

In [9]:
kwargs = {"min_length": 256, "max_length": 256}
data_generator_train = ConstantLengthDataGenerator(
    X_train, y_train, batch_size=batch_size, **kwargs
)
data_generator_val = ConstantLengthDataGenerator(
    X_val, y_val, batch_size=len(y_val), **kwargs
)

In [ ]:
history = model.fit(
    data_generator_train, epochs=5, validation_data=next(data_generator_val)
)

Epoch 1/5
50/50 [==============================] - 20s 374ms/step - loss: 1.9065 - accuracy: 0.2559 - val_loss: 2.0819 - val_accuracy: 0.1316
Epoch 2/5
22/50 [============>.................] - ETA: 9s - loss: 1.8294 - accuracy: 0.3075 

In [ ]:
metrics = model.evaluate(data_generator_val, return_dict=True)
mlflow.log_metrics(metrics)

In [ ]:
figure = plt.figure()
plt.plot(history.history["accuracy"])
plt.plot(history.history["val_accuracy"])
plt.title("model accuracy")
plt.ylabel("accuracy")
plt.xlabel("epoch")
plt.legend(["train", "validation"], loc="upper left")
figure

In [ ]:
mlflow.log_figure(figure, "data/figures/acc.png")

In [ ]:
figure = plt.figure()
plt.plot(history.history["loss"])
plt.plot(history.history["val_loss"])
plt.title("model loss")
plt.ylabel("loss")
plt.xlabel("epoch")
plt.legend(["train", "validation"], loc="upper left")
figure

In [ ]:
mlflow.log_figure(figure, "data/figures/loss.png")

In [ ]:
mlflow.end_run()